In [1]:
import json
from tqdm import tqdm

In [2]:
# !wget https://huseinhouse-storage.s3-ap-southeast-1.amazonaws.com/bert-bahasa/dumping-twitter-6-july-2019.json
# !wget https://malaya-dataset.s3-ap-southeast-1.amazonaws.com/dumping/twitter/2020-02-22-twitter-dump-in.json
# !wget https://malaya-dataset.s3-ap-southeast-1.amazonaws.com/dumping/twitter/2020-03-08-twitter-dump-in.json
# !wget https://huseinhouse-storage.s3-ap-southeast-1.amazonaws.com/bert-bahasa/dumping-instagram-6-july-2019.json

In [3]:
with open('dumping-instagram-6-july-2019.json') as fopen:
    instagram = json.load(fopen)
    
len(instagram)

695571

In [4]:
with open('dumping-twitter-6-july-2019.json') as fopen:
    twitter = json.load(fopen)
    
len(twitter)

6597867

In [5]:
files = ['2020-02-22-twitter-dump-in.json', '2020-03-08-twitter-dump-in.json']

for file in files:
    print(file)

    with open(file) as fopen:
        temp = json.load(fopen)

    for i in tqdm(range(len(temp))):
        retweet = temp[i]['retweet_text_full']
        t = temp[i]['data_text']
        if retweet != 'NULL' and len(retweet) > len(t):
            t = retweet
        twitter.append(t)

2020-02-22-twitter-dump-in.json


100%|██████████| 1617795/1617795 [00:01<00:00, 1419559.76it/s]


2020-03-08-twitter-dump-in.json


100%|██████████| 1711555/1711555 [00:01<00:00, 1454402.12it/s]


In [6]:
import cleaning

In [7]:
twitter = twitter + instagram

In [8]:
import re

def preprocessing(string):
    string = re.sub(
        'http\S+|www.\S+',
        '',
        ' '.join(
            [i for i in string.split() if i.find('#') < 0 and i.find('@') < 0]
        ),
    )
    
    chars = ',.()!:\'"/;=-'
    for c in chars:
        string = string.replace(c, f' {c} ')
        
    string = re.sub(
        u'[0-9!@#$%^&*()_\-+{}|\~`\'";:?/.>,<]',
        ' ',
        string,
        flags = re.UNICODE,
    )
    string = re.sub(r'[ ]+', ' ', string).strip()
    
    return string.lower()

def loop(strings):
    for i in tqdm(range(len(strings))):
        strings[i] = preprocessing(strings[i])
    return strings

In [9]:
%%time

twitter = cleaning.multiprocessing(twitter, loop)

100%|██████████| 663924/663924 [00:46<00:00, 14134.19it/s]


CPU times: user 7.6 s, sys: 6.49 s, total: 14.1 s
Wall time: 58.5 s


In [10]:
%%time

temp_vocab = list(set(cleaning.multiprocessing(twitter, cleaning.unique_words)))

CPU times: user 5.01 s, sys: 2.89 s, total: 7.9 s
Wall time: 16.9 s


In [11]:
%%time
temp_dict = cleaning.multiprocessing(temp_vocab, cleaning.duplicate_dots_marks_exclamations, list_mode = False)
print(len(temp_dict))

0
CPU times: user 372 ms, sys: 1.14 s, total: 1.51 s
Wall time: 3.08 s


In [12]:
%%time

twitter = cleaning.multiprocessing_multiple(twitter, temp_dict, cleaning.string_dict_cleaning)

100%|██████████| 663924/663924 [00:11<00:00, 59348.45it/s]


CPU times: user 6.12 s, sys: 5.07 s, total: 11.2 s
Wall time: 21.3 s


In [13]:
%%time
temp_dict = cleaning.multiprocessing(temp_vocab, cleaning.remove_underscore, list_mode = False)
print(len(temp_dict))

0
CPU times: user 344 ms, sys: 1.24 s, total: 1.59 s
Wall time: 1.93 s


In [14]:
%%time

twitter = cleaning.multiprocessing_multiple(twitter, temp_dict, cleaning.string_dict_cleaning)

100%|██████████| 663924/663924 [00:11<00:00, 60246.62it/s]


CPU times: user 6.13 s, sys: 5.45 s, total: 11.6 s
Wall time: 21.6 s


In [15]:
%%time
temp_dict = cleaning.multiprocessing(temp_vocab, cleaning.isolate_spamchars, list_mode = False)
print(len(temp_dict))

13
CPU times: user 313 ms, sys: 1.27 s, total: 1.58 s
Wall time: 1.98 s


In [16]:
%%time

twitter = cleaning.multiprocessing_multiple(twitter, temp_dict, cleaning.string_dict_cleaning)

100%|██████████| 663924/663924 [00:11<00:00, 57841.53it/s]


CPU times: user 6.16 s, sys: 5.05 s, total: 11.2 s
Wall time: 21.4 s


In [17]:
%%time
temp_dict = cleaning.multiprocessing(temp_vocab, cleaning.break_short_words, list_mode = False)
print(len(temp_dict))

0
CPU times: user 303 ms, sys: 1.41 s, total: 1.72 s
Wall time: 1.97 s


In [18]:
%%time

twitter = cleaning.multiprocessing_multiple(twitter, temp_dict, cleaning.string_dict_cleaning)

100%|██████████| 663924/663924 [00:11<00:00, 58795.47it/s]


CPU times: user 6.3 s, sys: 4.94 s, total: 11.2 s
Wall time: 21.8 s


In [19]:
%%time
temp_dict = cleaning.multiprocessing(temp_vocab, cleaning.break_long_words, list_mode = False)
print(len(temp_dict))

0
CPU times: user 317 ms, sys: 1.31 s, total: 1.63 s
Wall time: 1.89 s


In [20]:
%%time

twitter = cleaning.multiprocessing_multiple(twitter, temp_dict, cleaning.string_dict_cleaning)

100%|██████████| 663924/663924 [00:12<00:00, 53336.13it/s]


CPU times: user 6.72 s, sys: 4.77 s, total: 11.5 s
Wall time: 24.5 s


In [21]:
%%time
temp_dict = cleaning.multiprocessing(temp_vocab, cleaning.remove_ending_underscore, list_mode = False)
print(len(temp_dict))

0
CPU times: user 311 ms, sys: 1.39 s, total: 1.7 s
Wall time: 1.92 s


In [22]:
%%time

twitter = cleaning.multiprocessing_multiple(twitter, temp_dict, cleaning.string_dict_cleaning)

100%|██████████| 663924/663924 [00:11<00:00, 60094.51it/s]


CPU times: user 6.29 s, sys: 5.14 s, total: 11.4 s
Wall time: 21.7 s


In [23]:
%%time
temp_dict = cleaning.multiprocessing(temp_vocab, cleaning.remove_starting_underscore, list_mode = False)
print(len(temp_dict))

0
CPU times: user 328 ms, sys: 1.29 s, total: 1.62 s
Wall time: 1.8 s


In [24]:
%%time

twitter = cleaning.multiprocessing_multiple(twitter, temp_dict, cleaning.string_dict_cleaning)

100%|██████████| 663924/663924 [00:11<00:00, 59968.21it/s]


CPU times: user 6.16 s, sys: 4.72 s, total: 10.9 s
Wall time: 21.1 s


In [25]:
%%time
temp_dict = cleaning.multiprocessing(temp_vocab, cleaning.end_punct, list_mode = False)
print(len(temp_dict))

7786
CPU times: user 352 ms, sys: 1.41 s, total: 1.76 s
Wall time: 2.03 s


In [26]:
%%time

twitter = cleaning.multiprocessing_multiple(twitter, temp_dict, cleaning.string_dict_cleaning)

100%|██████████| 663924/663924 [00:11<00:00, 55675.14it/s]


CPU times: user 6.15 s, sys: 4.88 s, total: 11 s
Wall time: 22.1 s


In [27]:
%%time
temp_dict = cleaning.multiprocessing(temp_vocab, cleaning.start_punct, list_mode = False)
print(len(temp_dict))

8141
CPU times: user 368 ms, sys: 1.3 s, total: 1.66 s
Wall time: 1.93 s


In [28]:
%%time

twitter = cleaning.multiprocessing_multiple(twitter, temp_dict, cleaning.string_dict_cleaning)

100%|██████████| 663924/663924 [00:12<00:00, 54774.59it/s]


CPU times: user 5.9 s, sys: 4.8 s, total: 10.7 s
Wall time: 21.9 s


In [29]:
%%time
temp_dict = cleaning.multiprocessing(temp_vocab, cleaning.join_dashes, list_mode = False)
print(len(temp_dict))

0
CPU times: user 320 ms, sys: 1.42 s, total: 1.74 s
Wall time: 2 s


In [30]:
%%time

twitter = cleaning.multiprocessing_multiple(twitter, temp_dict, cleaning.string_dict_cleaning)

100%|██████████| 663924/663924 [00:11<00:00, 58428.04it/s]


CPU times: user 6.08 s, sys: 4.87 s, total: 10.9 s
Wall time: 21.7 s


In [31]:
twitter = [s.lower() for s in twitter]

In [32]:
twitter[:100]

['ternyata kl lg sdih bisa ngasilin makanan enak',
 'abu kampret',
 'bapa saya suka pake oppo saya sukanya nokia kaka saya sukanya samsung yg penting punya hape aja',
 'ngelamar kasih cincin tp kok mukanya songong ya sedih gue liatnya',
 'caption iki nggarai uwong males nikah min ya kali manusia arep punah ngunu neg gak nikah',
 'pertanyaannya sederhana jika kami memang dukung prabowo ngapain selama kampanye kemarin capek dukung jokowi sampa',
 '',
 'memiliki sedikit iman lebih berharga dari pada memiliki segudang emas',
 'untuk mengamankan suara partai ahmad rofiq selaku sekjen partai perindo meminta kepada seluruh caleg dan struktur',
 'dom jakpus sih bebas mau ketemuan or shopee',
 'bisa dapet duit ini kaga punya mobil juga kan kaya gemer gemer ini kaga',
 'on jadahnyaaa in sorry bad english hihuheheho',
 'valentino rossi tidak setuju kompetisi motogp dimulai dari eropa',
 'sis tak faham apa yang mungkin ini puncanya tu',
 'martabak terang bulan martabak untuk yg asin gurih a k a ma

In [33]:
with open('ms-socialmedia.json', 'w') as fopen:
    json.dump(twitter, fopen)

In [34]:
# !wget https://malaya-dataset.s3-ap-southeast-1.amazonaws.com/dumping/twitter/2020-02-22-twitter-dump-en.json
# !wget https://malaya-dataset.s3-ap-southeast-1.amazonaws.com/dumping/twitter/2020-03-08-twitter-dump-en.json

In [35]:
files = ['2020-02-22-twitter-dump-en.json', '2020-03-08-twitter-dump-en.json']

twitter_en = []

for file in files:
    print(file)

    with open(file) as fopen:
        temp = json.load(fopen)

    for i in tqdm(range(len(temp))):
        retweet = temp[i]['retweet_text_full']
        t = temp[i]['data_text']
        if retweet != 'NULL' and len(retweet) > len(t):
            t = retweet
        twitter_en.append(t)

2020-02-22-twitter-dump-en.json


100%|██████████| 717858/717858 [00:00<00:00, 1297407.09it/s]


2020-03-08-twitter-dump-en.json


100%|██████████| 716237/716237 [00:00<00:00, 1164060.63it/s]


In [36]:
%%time

twitter_en = cleaning.multiprocessing(twitter_en, loop)

100%|██████████| 89630/89630 [00:02<00:00, 41277.36it/s]


CPU times: user 1.04 s, sys: 1.81 s, total: 2.85 s
Wall time: 4.77 s


In [37]:
twitter_en = [s.lower() for s in twitter_en]
twitter_en = list(filter(None, twitter_en))

In [38]:
with open('en-socialmedia.json', 'w') as fopen:
    json.dump(twitter_en, fopen)